In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from utility import analysis_parameters as ap
import numpy as np
import dask
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [2]:
model="SCREAM"
var_name="rlt"

# Global tropical cloud ice


In [4]:
chunk_dict = {"time":3936//6, "lat":240, "lon":1440}
var = xr.open_dataset(ap.get_var_file(model, var_name), chunks=chunk_dict)[var_name]
var

<xarray.DataArray 'rlt' (time: 3936, lat: 240, lon: 1440)>
dask.array<open_dataset-d7b5a2b0fc3c89b6bca2d06ea0d9c4acrlt, shape=(3936, 240, 1440), dtype=float32, chunksize=(656, 240, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2020-01-20 00:00:00 ... 2020-03-01 23:45:00
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * lat      (lat) float64 -29.88 -29.62 -29.38 -29.12 ... 29.38 29.62 29.88
Attributes:
    standard_name:      toa_net_downward_longwave_flux
    long_name:          Net longwave flux at top of model
    units:              W m-2
    Sampling_Sequence:  rad_lwsw
    origin:             FLNT

In [ ]:
if var_name[:2]=="rl":
    bins = np.linspace(80,310,40)
elif var_name[:2]=="rs":
    bins = np.linspace(-5,1300,40)
elif var_name=="clivi":
    bins = np.logspace(-7,1,40) 


In [ ]:
hist, edges = dask.array.histogram(var, bins=bins, range=[0,1300]) # range=[1e-7, 10], [70,320], [0,1300]
counts = hist.compute()


In [ ]:
counts

In [ ]:
## Histogram ##
x = (edges[1:]+edges[:-1])/2 # np.linspace(-7,1)
plt.bar(x, counts, width=(bins[-1]-bins[0])/40) # np.array(np.logspace(-7,1,40))[:-1]
# plt.xscale("log")
# plt.xlim([80,310])
if var_name[0]=="r":
    plt.xlabel(" (W/m$^2$)")
elif var_name[:3]=="cli":
    plt.xlabel("IWP (kg/m$^2$)")
plt.ylabel("Count")
plt.title(model+" "+var_name)
plt.savefig("../plots/"+model+"_"+var_name+"_ITCZ.png")
plt.show()
